## Feature Engineering

In [ ]:
import numpy as np
import pandas as pd

Now, we want to add more features for enhanced predictability. To start, we will add a binary predictor to indicate whether the location is "null" in the corresponding observation.

In [ ]:
# add binary predictor to indicate whether location is missing
#train_data['location_null'] = train_data['location'].isnull().astype(int)
#test_data['location_null'] = test_data['location'].isnull().astype(int)